# SPACEX Falcon 9 Visual Analytics with Folium

In this notebook we will try visualize the geographic locations of the Launch Sites

### Imort the Required Libraries

In [5]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

### Task 1. Mark all launch sites on a map

#### Read the dataset

In [6]:
df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [7]:
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long',]]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


#### Create our Folium Map for Visualization

In [10]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
m = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
icon = DivIcon(
    icon_size=(20, 20),
    icon_anchor=(0,0),
    html= '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
)
)
m.add_child(circle)
m.add_child(marker)

In [12]:
# Launch sites
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [20]:
for lat, lng, site in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    circle2 = folium.Circle(location = [lat, lng], radius=1000, color='#d35400', fill = True).add_child(folium.Popup(site))

    marker2 = folium.map.Marker(location = [lat, lng],
                              icon=DivIcon(
                                  icon_size=(20,20),
                                  icon_anchor=(0,0),
                                  html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'LAUNCH SITE'
                              ))
    m.add_child(circle2)
    m.add_child(marker2)

m

#### 2. Mark the Success/failed launches for each site on the map

In [25]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.head()
#launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
#launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long',]]
#launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [26]:
marker_cluster = MarkerCluster()

We shall create a function that checks the value in class column and updates a new column Color with either red for 0 and green for 1

In [40]:
def success(dataframe, column):
    color = ['red' if value == 0 else 'green' for value in column]
    dataframe['color'] = color
    return dataframe.head()
            

In [41]:
column = spacex_df['class']
success(spacex_df, column)

C:\Users\Carlt\AppData\Local\Temp\ipykernel_30404\2508115481.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['color'] = color


,Launch Site,Lat,Long,class,color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [42]:
spacex_df.tail()

,Launch Site,Lat,Long,class,color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


We will then add the icons indicating the number of success and failure rates for each launch station using the marker cluster


In [50]:
m.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    marker=folium.Marker(location=[record['Lat'], record['Long']], 
                         icon = folium.Icon(color='white', icon_color = record['color']))
    marker_cluster.add_child(marker)
    
m  

#### Calculating the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [51]:
formatter= "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    seperator='Long:',
    empty_string='NaN',
    lng_first=False,
    prefix='Lat',
    num_digits=20,
    lat_formatter = formatter,
    lng_formatter=formatter
)

m.add_child(mouse_position)
m

We shall then write a function to calculate the distance between a launchsite and some areas of interest

In [62]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = round((R * c), 2)
    return str(distance) + ' '+ 'meters'

In [53]:
launchsite_lat = 34.63285
launchsite_lon= -120.60988
coast_lat = 34.56171
coast_lng=-120.62835

In [65]:
# Calculate distance between a launchsite and the coastline
coastline_distance = calculate_distance(launchsite_lat, launchsite_lon, coast_lat, coast_lng)

In [64]:
coastline_distance

'8.09 meters'

#### Draw a Polyline between a launch site to the selected coastline point

In [69]:
line = folium.PolyLine(locations = [[coast_lat, coast_lng], [launchsite_lat, launchsite_lon]], weight=1, popup=coastline_distance)
m.add_child(line)